In [1]:
''' 
 RUSBOOST BINARY
'''
import pandas as pd
def FoundFeatures(Path):
    # reading the found dataset for the training
    df = pd.read_excel(Path)
    Features = df.columns
    return Features

# removing the id and the subtypes of a given dataset
def extract_PATIENT_INFO(db):
    ids =  db.pop("ID")
    morf_codificata = db.pop("morf_codificata")
    luogoTC = db.pop("luogoTC_codificato")

    return db, ids, morf_codificata, luogoTC

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
import numpy as np

path = "Elements/Boruta_RusBoost/AcrossSplits/Split1_Boruta_RusBoost_ReduceDataset.xlsx"
cols = FoundFeatures(path)
len(cols)

OLD = pd.read_excel("Dataset/Cleaned_dataset.xlsx")
columns = ["ID", "morf_codificata",	"maligno",	"luogoTC_codificato"] + list(cols)
db = OLD[columns]  
db.shape


# extrating the label
y = db.pop("maligno")
X = db

X_train_folds = []
X_test_folds = []

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
# enumerate splits
outer_results = list()
best_params_per_fold = []
scores = pd.DataFrame(columns=['Outer Fold',"Grid-Search",'F1-score', 'Recall', 'Precision', 'Accuracy', 'Auc-Score', '#Features'])

for index, (train_ix, test_ix) in enumerate(cv_outer.split(X, y)):
    # split data
    X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    

    unique, counts = np.unique(y_test, return_counts=True)
    hist = dict(zip(unique, counts))
    print(f"Fold {index} - Test set class distribution: {hist}")
    X_test_outer = X_test
    X_test_folds.append(X_test)
    morf = X_test_outer["morf_codificata"]
    unique, counts = np.unique(morf, return_counts=True)
    hist = dict(zip(unique, counts))
    print(f"Fold {index} - Test set class distribution: {hist}")


    fold_df = pd.DataFrame(X_test)
    fold_df["label"] = y_test
    fold_df.to_csv(f"nested_CV/MinSet/fold_{index}_test_set.csv", index=False)


    fold_df = pd.DataFrame(X_train)
    X_train_folds.append(fold_df)
    fold_df["label"] = y_train
    fold_df.to_csv(f"nested_CV/MinSet/fold_{index}_train_set.csv", index=False)


Fold 0 - Test set class distribution: {0: 168, 1: 42}
Fold 0 - Test set class distribution: {0: 108, 1: 56, 2: 23, 3: 15, 4: 4, 5: 4}
Fold 1 - Test set class distribution: {0: 168, 1: 41}
Fold 1 - Test set class distribution: {0: 116, 1: 46, 2: 21, 3: 8, 4: 12, 5: 6}
Fold 2 - Test set class distribution: {0: 167, 1: 42}
Fold 2 - Test set class distribution: {0: 134, 1: 29, 2: 16, 3: 14, 4: 12, 5: 4}
Fold 3 - Test set class distribution: {0: 167, 1: 42}
Fold 3 - Test set class distribution: {0: 118, 1: 46, 2: 18, 3: 13, 4: 11, 5: 3}
Fold 4 - Test set class distribution: {0: 167, 1: 42}
Fold 4 - Test set class distribution: {0: 114, 1: 51, 2: 22, 3: 10, 4: 10, 5: 2}


In [74]:
path = "Elements/Boruta_RusBoost/AcrossSplits/Split1_Boruta_RusBoost_ReduceDataset.xlsx"
cols = FoundFeatures(path)
len(cols)

60

In [75]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance

def loading_importances(rusboost_model, X_test, y_test, all_importances):
    # Compute permutation importance 
    result = permutation_importance(rusboost_model, X_test, y_test, n_repeats=300, random_state=42, n_jobs=8, scoring="f1")
    feature_importances = result.importances_mean
    
    # Store feature importances for the current split
    all_importances.append(feature_importances)
    # return the appended list with the current mean importances for each feature
    return all_importances

In [76]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_mean_importances(all_importances, feature_names):
    # Convert list of importances into a numpy array
    importances_array = np.array(all_importances)

    # Calculate the median importances across all splits/iterations
    median_importances = np.median(importances_array, axis=0)

    # Sort the median importances after calculation
    sorted_indices = np.argsort(median_importances)[::-1]
    sorted_median_importances = median_importances[sorted_indices]
    sorted_features = np.array(feature_names)[sorted_indices]

    # Print information about features with mean importance less than 0
    features_below_zero = sorted_features[sorted_median_importances < 0]
    importances_below_zero = sorted_median_importances[sorted_median_importances < 0]

    print("Features with median importance less than 0:")
    for feature, importance in zip(features_below_zero, importances_below_zero):
        print(f"{feature}: {importance}")

    # Plot the median importances across all splits/iterations
    plt.figure(figsize=(20, 10))
    plt.plot(
        sorted_features,
        sorted_median_importances,
        marker='o',
        linestyle='-',
        color=sns.color_palette("autumn", 1)[0],
        markersize=8,
    )



    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(fontsize=15)

    # Set y-axis limits with padding
    y_min = min(sorted_median_importances) - 0.01
    y_max = max(sorted_median_importances) + 0.01
    plt.ylim(y_min, y_max)
    plt.xlabel('Feature Names', fontsize=16, labelpad=10)
    plt.ylabel('Median Importance Score', fontsize=16, labelpad=10)
    plt.title(f'Median Feature Importances from RUSBoostClassifier', fontsize=20, pad=15)
    plt.tight_layout()

    # Save the plot for the median importances
    plt.savefig('Elements/RusBoost/MinimumSet/Median_feature_importances_line_plot.png', dpi=300, bbox_inches='tight')
    plt.close()

    return sorted_features, sorted_median_importances


In [143]:
import numpy as np
import matplotlib.pyplot as plt

def plot_line_plots_raw_values(all_importances, feature_names):
    importances_array = np.array(all_importances)
    num_folds = importances_array.shape[0]
    folds = np.arange(1, num_folds + 1)

    # Use median instead of mean to rank feature importance
    median_importances = np.median(importances_array, axis=0)
    sorted_indices = np.argsort(median_importances)[::-1]

    top_5_indices = sorted_indices[:5]
    bottom_5_indices = sorted_indices[-5:]

    fig, axes = plt.subplots(1, 2, figsize=(24, 12))
    fig.subplots_adjust(bottom=0.3, wspace=0.3)

    tab10_colors = [plt.cm.tab10(i) for i in range(10)]
    top_colors = tab10_colors[:5]
    bottom_colors = tab10_colors[5:]

    marker_styles = ['o', 'o', 'o', 'o', 'o']

    def style_plot(ax, indices, title, color_palette):
        features = np.array(feature_names)[indices]
        x = folds

        y_data_for_plot = importances_array[:, indices]
        y_min_local = np.min(y_data_for_plot)
        y_max_local = np.max(y_data_for_plot)
        y_range_local = y_max_local - y_min_local

        if y_range_local == 0:
            y_min_local -= 0.01
            y_max_local += 0.01
        else:
            padding = y_range_local * 0.15
            y_min_local -= padding
            y_max_local += padding
        ax.set_ylim(y_min_local, y_max_local)

        legend_labels = []

        for i, idx in enumerate(indices):
            y = importances_array[:, idx]
            median_val = np.median(y)
            legend_labels.append(f"{features[i]} (median={median_val:.3f})")

            ax.plot(
                x, y,
                color=color_palette[i],
                linewidth=3,
                marker=marker_styles[i % len(marker_styles)],
                markersize=10,
                markerfacecolor='white',
                markeredgewidth=2
            )

        ax.set_title(title, fontsize=20, fontweight='bold')
        ax.set_xlabel('Fold', fontsize=17, fontweight='bold')
        if ax == axes[0]:
            ax.set_ylabel('Feature Importance', fontsize=17, fontweight='bold')

        ax.set_xticks(folds)
        ax.set_xlim(folds[0] - 0.3, folds[-1] + 1.2)
        ax.grid(axis='y', linestyle='--', alpha=0.7)
        ax.tick_params(axis='both', which='major', labelsize=15, direction='in', top=True, right=True)
        ax.xaxis.set_ticks_position('both')
        ax.yaxis.set_ticks_position('both')
        ax.spines['top'].set_visible(True)
        ax.spines['right'].set_visible(True)

        ax.legend(
            legend_labels,
            loc='upper center',
            bbox_to_anchor=(0.5, -0.3),
            fontsize=18,
            ncol=1,
            frameon=True,
            fancybox=True,
            shadow=True
        )

    style_plot(axes[0], top_5_indices, 'Top 5 Features Importance Across Outer Folds', top_colors)
    style_plot(axes[1], bottom_5_indices, 'Bottom 5 Features Importance Across Outer Folds', bottom_colors)

    plt.tight_layout(rect=[0, 0.1, 1, 1])
    plt.savefig('Elements/RusBoost/MinimumSet/Top_and_Bottom_5_features_raw_values_line_plots_reimproved.png', dpi=300)
    plt.close()


In [144]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def plot_box_plots_for_best_and_worst_features(all_importances, feature_names):
    # Convert list of importances into a numpy array
    importances_array = np.array(all_importances)

    # Calculate statistics
    mean_importances = importances_array.mean(axis=0)
    median_importances = np.median(importances_array, axis=0)
    std_importances = np.std(importances_array, axis=0)
    min_importances = importances_array.min(axis=0)
    max_importances = importances_array.max(axis=0)

    # Sort indices based on median importances
    sorted_indices = np.argsort(median_importances)[::-1]
    sorted_features = np.array(feature_names)[sorted_indices]
    sorted_mean_importances = mean_importances[sorted_indices]
    sorted_median_importances = median_importances[sorted_indices]
    sorted_std_importances = std_importances[sorted_indices]
    sorted_min_importances = min_importances[sorted_indices]
    sorted_max_importances = max_importances[sorted_indices]

    # Get top 5 and last 5 features
    top_5_indices = sorted_indices[:5]
    bottom_5_indices = sorted_indices[-5:]

    # Extract importance values
    top_5_importances_values = importances_array[:, top_5_indices]
    bottom_5_importances_values = importances_array[:, bottom_5_indices]

    def create_legend_labels(indices):
        return [
            f"{sorted_features[i]}:\nMean={sorted_mean_importances[i]:.4f}, "
            f"Median={sorted_median_importances[i]:.4f}, Std={sorted_std_importances[i]:.4f}, "
            f"Min={sorted_min_importances[i]:.4f}, Max={sorted_max_importances[i]:.4f}"
            for i in indices
        ]

    # Create labels for legends
    top_5_legend_labels = create_legend_labels(range(5))
    bottom_5_legend_labels = create_legend_labels(range(-5, 0))

    def plot_features_boxplot(importances, labels, title, save_path):
        plt.figure(figsize=(10, 6))
        sns.boxplot(data=importances, orient='h', palette="coolwarm")
        plt.xlabel('Decrease in F1-score on the Test Set', fontsize=14)
        plt.title(title, fontsize=16)

        # Create horizontal legend below the plot
        plt.legend(
            labels,
            loc='upper center',
            bbox_to_anchor=(0.5, -0.15),
            ncol=1,
            frameon=False,
            fontsize=10,
        )
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()

    # Plot for top 5 features
    plot_features_boxplot(
        top_5_importances_values,
        top_5_legend_labels,
        'Box Plot of Top 5 Features by Importance',
        'Elements/RusBoost/MinimumSet/Top_5_features_box_plot.png',
    )

    # Plot for bottom 5 features
    plot_features_boxplot(
        bottom_5_importances_values,
        bottom_5_legend_labels,
        'Box Plot of Bottom 5 Features by Importance',
        'Elements/RusBoost/MinimumSet/Bottom_5_features_box_plot.png',
    )


In [145]:
import pandas as pd
OLD = pd.read_excel("Dataset/Cleaned_dataset.xlsx")
OLD.shape

(1046, 128)

In [146]:
cols

Index(['BAS_MORPHOLOGICAL_VolumeIBSIRNU0',
       'BAS_MORPHOLOGICAL_ApproximateVolumeIBSIYEKZ',
       'BAS_MORPHOLOGICAL_voxelsCountingIBSINo',
       'BAS_MORPHOLOGICAL_SurfaceAreaIBSIC0JK',
       'BAS_MORPHOLOGICAL_SurfaceToVolumeRatioIBSI2PR5',
       'BAS_MORPHOLOGICAL_CompacityIBSINo',
       'BAS_MORPHOLOGICAL_Maximum3DDiameterIBSIL0JK',
       'BAS_MORPHOLOGICAL_IntegratedIntensityIBSI99N0',
       'BAS_INTENSITYBASED_MeanHUIBSIQ4LE',
       'BAS_INTENSITYBASED_SkewnessHUIBSIKE2A',
       'BAS_INTENSITYBASED_KurtosisHUIBSIIPH6',
       'BAS_INTENSITYBASED_MedianHUIBSIY12H',
       'BAS_INTENSITYBASED_MinimumGreyLevelHUIBSI1GSF',
       'BAS_INTENSITYBASED_10thPercentileHUIBSIQG58',
       'BAS_INTENSITYBASED_25thPercentileHUIBSINo',
       'BAS_INTENSITYBASED_50thPercentileHUIBSIY12H',
       'BAS_INTENSITYBASED_75thPercentileHUIBSINo',
       'BAS_INTENSITYBASED_90thPercentileHUIBSI8DWT',
       'BAS_INTENSITYBASED_QuartileCoefficientOfDispersionHUIBSI9S40',
       'BAS_INTE

In [147]:
columns = ["ID", "morf_codificata",	"maligno",	"luogoTC_codificato"] + list(cols)
db = OLD[columns]  # Convert `cols` to a list if it's a set
db.shape

(1046, 64)

In [148]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



# Remove non-numeric columns (if necessary) because correlation is only defined for numeric features
db_numeric = db.select_dtypes(include=['number'])

# Compute the correlation matrix
corr_matrix = db_numeric.corr()


# Optionally, set a threshold for highly correlated features (e.g., above 0.9)
threshold = 0.9
high_corr_pairs = [(i, j) for i in corr_matrix.columns for j in corr_matrix.columns if abs(corr_matrix.loc[i, j]) > threshold and i != j]

# Print pairs of highly correlated features
print("Highly correlated feature pairs (correlation > 0.9):")
print(len(high_corr_pairs))

Highly correlated feature pairs (correlation > 0.9):
400


In [149]:
db.head()

,ID,morf_codificata,maligno,luogoTC_codificato,BAS_MORPHOLOGICAL_VolumeIBSIRNU0,BAS_MORPHOLOGICAL_ApproximateVolumeIBSIYEKZ,BAS_MORPHOLOGICAL_voxelsCountingIBSINo,BAS_MORPHOLOGICAL_SurfaceAreaIBSIC0JK,BAS_MORPHOLOGICAL_SurfaceToVolumeRatioIBSI2PR5,BAS_MORPHOLOGICAL_CompacityIBSINo,...,BAS_GLRLM_RunLengthNonUniformityIBSIW92Y,BAS_NGTDM_CoarsenessIBSIQCDE,BAS_NGTDM_ContrastIBSI65HE,BAS_NGTDM_BusynessIBSINQ30,BAS_NGTDM_StrengthIBSI1X9X,BAS_GLSZM_LowGrayLevelZoneEmphasisIBSIXMSY,BAS_GLSZM_HighGrayLevelZoneEmphasisIBSI5GN9,BAS_GLSZM_GreyLevelNonUniformityIBSIJNSA,BAS_GLSZM_ZonePercentageIBSIP30P,BAS_GLSZM_ZoneSizeVarianceIBSI3NSA
0,RADIO-BOL_1,0,0,1,880.755759,552.0,552,663.029263,0.752796,1.328381,...,332.860305,0.014297,0.053627,0.049563,0.868827,0.000101,9913.069444,7.416667,0.130435,473.472222
1,RADIO-BOL_2,0,0,1,683.096680,1050.0,1050,481.235545,0.704491,1.419464,...,758.450326,0.006197,0.084300,0.091586,0.590565,0.000105,9595.422222,18.271111,0.214286,238.328889
2,RADIO-BOL_3,1,0,1,2210.932693,1876.0,1876,1081.385004,0.489108,2.044538,...,1086.638973,0.004222,0.039441,0.156383,0.235307,0.000095,10531.566667,17.680000,0.079957,2890.329956
3,RADIO-BOL_4,0,0,1,15119.932175,5520.0,5520,5734.182426,0.379247,2.636807,...,2784.748239,0.001720,0.030624,0.281560,0.209275,0.000107,9398.774908,23.590406,0.049094,13270.860146
4,RADIO-BOL_6,1,0,1,6990.971956,5894.0,5894,2204.151035,0.315285,3.171730,...,3434.212603,0.001597,0.029569,0.190679,0.525746,0.000102,9932.738983,40.844068,0.100102,5249.145659


In [150]:
from numpy import mean
from numpy import std
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from Models import RusBoost
from sklearn.metrics import  accuracy_score, confusion_matrix,precision_score, recall_score, f1_score,roc_auc_score,roc_curve,precision_recall_curve, average_precision_score

import numpy as np
from sklearn.inspection import permutation_importance

 # extrating the label
y = db.pop("maligno")
X = db

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
# enumerate splits
outer_results = list()
best_params_per_fold = []
scores = pd.DataFrame(columns=['Outer Fold',"Grid-Search",'F1-score', 'Recall', 'Precision', 'Accuracy', 'Auc-Score', '#Features'])


folds_train = []
folds_test = []
all_importances = []

for index, (train_ix, test_ix) in enumerate(cv_outer.split(X, y)):
	# split data
	X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
	y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]


	folds_train.append(X_train.copy())
	folds_test.append(X_test.copy())

	# configure the cross-validation procedure
	cv_inner = StratifiedKFold(n_splits=5,shuffle = True, random_state=42)
	
	pipeline = RusBoost.Define_Pipeline()
	X_train, ids, morf_codificata, luogoTC_train = extract_PATIENT_INFO(X_train)
	Grid_searchOBJ = RusBoost.Grid_Search(pipeline=pipeline, cv = cv_inner, X_train=X_train, y_train=y_train)

	best_params = Grid_searchOBJ.best_params_
	best_params_per_fold.append(Grid_searchOBJ.best_params_)
	print(f"Fold {len(best_params_per_fold)}: Best Params = {Grid_searchOBJ.best_params_}")

	Chosen_pipeline = RusBoost.Define_Pipeline()
	Chosen_pipeline.set_params(**Grid_searchOBJ.best_params_) 
	print(X_train.shape)
	fitted_pipeline = RusBoost.fit(pipeline=Chosen_pipeline, X_train=X_train, y_train=y_train, Prefix="MinimumSet/Split{}_".format(index))


	rusboost_model = fitted_pipeline['rusboostclassifier']
	# Define a prediction function for the model
	X_test, ids_test, morf_codificata_test, luogoTC_test  = extract_PATIENT_INFO(X_test)


	predictions = fitted_pipeline.predict(X_test)
	probs = fitted_pipeline.predict_proba(X_test)

	###Metrics
	Accuracy = round(accuracy_score(y_test, predictions),3)
	precision = round(precision_score(y_test, predictions),3)
	recall = round(recall_score(y_test, predictions),3)
	f1 = round(f1_score(y_test, predictions),3)
	print("F1 Test is", f1)
	probs = probs[:, 1]
	auc_roc = round(roc_auc_score(y_test,probs),3)

	##Features
	AllFeatures_RusBoost = FoundFeatures("./Elements/RusBoost/MinimumSet/Split{}_RusBoost_ReduceDataset.xlsx".format(index))
	print("it has found {} features".format(len(AllFeatures_RusBoost)))
	
	new_row = [index,Grid_searchOBJ.best_score_, f1, recall, precision, Accuracy, auc_roc, len(AllFeatures_RusBoost)]  
	scores.loc[index] = new_row  # Append the new row

	all_importances = loading_importances(rusboost_model, X_test, y_test, all_importances)
features_names = X_train.columns
calculate_mean_importances(all_importances, features_names)
plot_box_plots_for_best_and_worst_features(all_importances, features_names)
plot_line_plots_raw_values(all_importances, features_names)
	
	

	

	
    
scores["Outer Fold"] = scores["Outer Fold"].astype(int)
scores["#Features"] = scores["#Features"].astype(int)
scores.to_csv('Elements/RusBoost/MinimumSet/scores.csv', index=False)  # index=False to avoid saving the row numbers
scores

[2025-07-08 15:07:12,949: INFO: RusBoost: Defining the pipeline...]
[2025-07-08 15:07:12,951: INFO: RusBoost: Let's define the cross validation:StratifiedKFold(n_splits=5, random_state=42, shuffle=True)]
[2025-07-08 15:07:12,952: INFO: RusBoost: Let's define the parameters for the grid search: {'rusboostclassifier__n_estimators': [150, 300], 'rusboostclassifier__learning_rate': [0.05, 0.01]}]
Best: 0.795299 using {'rusboostclassifier__learning_rate': 0.05, 'rusboostclassifier__n_estimators': 150}
Fold 1: Best Params = {'rusboostclassifier__learning_rate': 0.05, 'rusboostclassifier__n_estimators': 150}
[2025-07-08 15:07:19,066: INFO: RusBoost: Defining the pipeline...]
(836, 60)
Time taken to fit the model: 0.50553 seconds
[2025-07-08 15:07:20,258: INFO: RusBoost: saving the resulting dataframe into: RusBoost_ReduceDataset.xlsx]
F1 Test is 0.768
it has found 60 features
[2025-07-08 15:08:06,549: INFO: RusBoost: Defining the pipeline...]
[2025-07-08 15:08:06,552: INFO: RusBoost: Let's de

,Outer Fold,Grid-Search,F1-score,Recall,Precision,Accuracy,Auc-Score,#Features
0,0,0.795299,0.768,0.905,0.667,0.890,0.963,60
1,1,0.806452,0.788,0.951,0.672,0.900,0.978,60
2,2,0.793478,0.848,0.929,0.780,0.933,0.969,60
3,3,0.805759,0.720,0.857,0.621,0.866,0.962,60
4,4,0.791857,0.833,0.952,0.741,0.923,0.967,60


In [13]:
folds_test[0].shape

(210, 63)

In [ ]:

Outer Fold	Grid-Search	F1-score	Recall	Precision	Accuracy	Auc-Score	#Features
0	0	0.795299	0.768	0.905	0.667	0.890	0.963	60
1	1	0.806452	0.788	0.951	0.672	0.900	0.978	60
2	2	0.793478	0.848	0.929	0.780	0.933	0.969	60
3	3	0.805759	0.720	0.857	0.621	0.866	0.962	60
4	4	0.791857	0.833	0.952	0.741	0.923	0.967	60

In [14]:
scores
	

,Outer Fold,Grid-Search,F1-score,Recall,Precision,Accuracy,Auc-Score,#Features
0,0,0.795299,0.768,0.905,0.667,0.890,0.963,60
1,1,0.806452,0.788,0.951,0.672,0.900,0.978,60
2,2,0.793478,0.848,0.929,0.780,0.933,0.969,60
3,3,0.805759,0.720,0.857,0.621,0.866,0.962,60
4,4,0.791857,0.833,0.952,0.741,0.923,0.967,60


In [15]:
best_params_per_fold 

[{'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 150},
 {'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 300},
 {'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 300},
 {'rusboostclassifier__learning_rate': 0.01,
  'rusboostclassifier__n_estimators': 300},
 {'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 300}]

In [16]:
import numpy as np
def compute_stats(scores):
    cols = scores.columns[1:-1]
    mean_row_values = scores[cols].mean()
    std_row_values = scores[cols].std()
    
    # Create new rows with NaNs for the columns that will be replaced
    mean_row = pd.Series("", index=scores.columns, name='mean')
    std_row = pd.Series("", index=scores.columns, name='std')
    mean_row[cols] = mean_row_values.round(3)
    std_row[cols] = std_row_values.round(3)
    
    # Convert Series to DataFrame
    mean_df = mean_row.to_frame().T
    std_df = std_row.to_frame().T
    
    scores_new = pd.concat([scores, mean_df, std_df], axis = 0)
    
    scores_new.loc["mean","#Features"] = str(int(round(np.mean(scores_new["#Features"][:-2]),0)))
    scores_new.loc["std","#Features"] = str(int(round(np.std( scores_new["#Features"][:-2]),0)))
    
    scores_new.to_csv('Elements/RusBoost/MinimumSet/scores.csv', index=False)  # index=False to avoid saving the row numbers

    return scores_new

scores_agg = compute_stats(scores)
scores_agg

,Outer Fold,Grid-Search,F1-score,Recall,Precision,Accuracy,Auc-Score,#Features
0,0,0.795299,0.768,0.905,0.667,0.89,0.963,60
1,1,0.806452,0.788,0.951,0.672,0.9,0.978,60
2,2,0.793478,0.848,0.929,0.78,0.933,0.969,60
3,3,0.805759,0.72,0.857,0.621,0.866,0.962,60
4,4,0.791857,0.833,0.952,0.741,0.923,0.967,60
mean,,0.799,0.791,0.919,0.696,0.902,0.968,60
std,,0.007,0.051,0.04,0.063,0.027,0.006,0
